In [1]:
! pip install gradio --quiet
! pip install opencv-python --quiet
! pip install pillow --quiet


]4;0;#000000\]4;1;#FFFFFF\]4;2;#CCCCCC\]4;3;#8f8f8f\]4;4;#FFFFFF\]4;5;#111111\]4;6;#CCCCCC\]4;7;#FFFFFF\]4;8;#404040\]4;9;#CCCCCC\]4;10;#FFFFFF\]4;11;#909090\]4;12;#CCCCCC\]4;13;#808080\]4;14;#CCCCCC\]4;15;#FFFFFF\]10;#FFFFFF\]11;[100]#000000\]12;#FFFFFF\]13;#FFFFFF\]17;#FFFFFF\]19;#000000\]4;232;#FFFFFF\]4;256;#FFFFFF\]708;[100]#000000\
]4;0;#000000\]4;1;#FFFFFF\]4;2;#CCCCCC\]4;3;#8f8f8f\]4;4;#FFFFFF\]4;5;#111111\]4;6;#CCCCCC\]4;7;#FFFFFF\]4;8;#404040\]4;9;#CCCCCC\]4;10;#FFFFFF\]4;11;#909090\]4;12;#CCCCCC\]4;13;#808080\]4;14;#CCCCCC\]4;15;#FFFFFF\]10;#FFFFFF\]11;[100]#000000\]12;#FFFFFF\]13;#FFFFFF\]17;#FFFFFF\]19;#000000\]4;232;#FFFFFF\]4;256;#FFFFFF\]708;[100]#000000\
]4;0;#000000\]4;1;#FFFFFF\]4;2;#CCCCCC\]4;3;#8f8f8f\]4;4;#FFFFFF\]4;5;#111111\]4;6;#CCCCCC\]4;7;#FFFFFF\]4;8;#404040\]4;9;#CCCCCC\]4;10;#FFFFFF\]4;11;#909090\]4;12;#CCCCCC\]4;13;#808080\]4;14

In [2]:
import cv2
import numpy as np
import os
import random
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import gradio as gr


def preprocess(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.normalize(img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    img = cv2.GaussianBlur(img, (5, 5), 0)
    img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    kernel = np.ones((3,3),np.uint8)
    img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    return img
    
def contour_segmentation(img):
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    # find the contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # find the largest contour
    max_area = 0
    max_contour = None
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > max_area:
            max_area = area
            max_contour = contour
    # find the bounding rectangle
    x, y, w, h = cv2.boundingRect(max_contour)
    # crop the image  into rect
    img = img[y:y+h, x:x+w]
    # img = img[y:y+(h if h < w else w), x:x+(h if h < w else w)]
    return img

def orientation(img):
    # return img if img.shape[0] < img.shape[1] else cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    return img 

def extract_features_orb(img, n_features=1000):
    orb = cv2.ORB_create(n_features)
    keypoints, descriptors = orb.detectAndCompute(img, None)
    return keypoints, descriptors

def extract_features_sift(img):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(img, None)
    return keypoints, descriptors

def extract_features_harris(img):
    # remove alpha channel
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = np.float32(gray)
    dst = cv2.cornerHarris(gray, 2, 3, 0.04)
    dst = cv2.dilate(dst, None)
    ret, dst = cv2.threshold(dst, 0.01 * dst.max(), 255, 0)
    dst = np.uint8(dst)
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    corners = cv2.cornerSubPix(gray, np.float32(centroids), (5, 5), (-1, -1), criteria)
    return corners

def bag_of_words(descriptors, kmeans):
    histogram = np.zeros(kmeans.n_clusters)
    if descriptors is not None:
        clusters = kmeans.predict(descriptors)
        for cluster in clusters:
            histogram[cluster] += 1
    return histogram

# additional more complex testing
# some very specific examples that are difficult to classify
# test_images, test_labels = load_images(data_test_dir)
# preprocessed_test_images = [preprocess(img) for img in test_images]
# aligned_test_images = [contour_segmentation(img) for img in preprocessed_test_images]
# oriented_test_images = [orientation(img) for img in aligned_test_images]
# # keypoints_test, descriptors_test = zip(*[extract_features_sift(img) for img in oriented_test_images])
# # keypoints_test, descriptors_test = zip(*[extract_features_orb(img) for img in oriented_test_images])
# descriptors_test = [extract_features_harris(img) for img in oriented_test_images]
# bovw_test = np.array([bag_of_words(d, kmeans) for d in descriptors_test])
# y_pred = rf.predict(bovw_test)
# print('Accuracy:', accuracy_score(test_labels, y_pred))

# y_pred = svm.predict(bovw_test)
# print('Accuracy:', accuracy_score(test_labels, y_pred))

# y_pred = knn.predict(bovw_test)
# print('Accuracy:', accuracy_score(test_labels, y_pred))

# # print oriented_test_images
# display(preprocessed_test_images, oriented_test_images, test_labels, y_pred)

# keypoints, descriptors = zip(*[extract_features_orb(img) for img in oriented])
# keypoints, descriptors = zip(*[extract_features_sift(img) for img in oriented])
# descriptors = [extract_features_harris(img) for img in oriented]
# keypoints = [extract_features_hog(img) for img in oriented]


/home/skywalker/code/University/COMP702/Comp702_Project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import gradio as gr
import os
import joblib

model_dir = "models"
feats = os.listdir(model_dir)
classies = ["svc", "rf", "knn"]
print(feats)

def load_models():
    models = {}
    for model in feats:
        models[f"{model}_kmeans".upper()] = joblib.load(os.path.join(model_dir, model, "kmeans.pkl"))
        for clas in classies:
            model_path = os.path.join(model_dir, model, f"{clas}.pkl")
            models[f"{model}_{clas}".upper()] = joblib.load(model_path)
    return models

models = load_models()
print(models)
def predict(model_name, model_feat_extract, img):
    img = preprocess(img)
    img = contour_segmentation(img)
    img = orientation(img)
    if model_feat_extract == "ORB":
        keypoints, descriptors = extract_features_orb(img)
    elif model_feat_extract == "SIFT":
        keypoints, descriptors = extract_features_sift(img)
    elif model_feat_extract == "HARRIS":
        descriptors = extract_features_harris(img)

    kmeans = models[f"{model_feat_extract}_KMEANS"]
    print("here")
    bovw = bag_of_words(descriptors, kmeans)
    model = models[f"{model_feat_extract}_{model_name}"]
    y_pred = model.predict([bovw])
    return y_pred[0], img
    

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    with gr.Row():
        model = gr.Dropdown(label="Classifier", choices=["SVC", "KNN", "RF"], interactive=True, value="RF")
        model_feat_extract = gr.Dropdown(label="Feat Extract", choices=["ORB", "SIFT", "HARRIS"], interactive=True, value="SIFT")
        prediction = gr.Label("")
    with gr.Row():
        img = gr.Image()
        out_img = gr.Image()
        

    # greet_btn.click(predict, [model, model_feat_extract, img] , [prediction, out_img])
    img.upload(predict, [model, model_feat_extract, img] , [prediction, out_img])
    model_feat_extract.select(predict, [model, model_feat_extract, img] , [prediction, out_img])
    model.select(predict, [model, model_feat_extract, img] , [prediction, out_img])

demo.launch(share=True)

['SIFT', 'ORB', 'HARRIS']
{'SIFT_KMEANS': MiniBatchKMeans(n_clusters=1000), 'SIFT_SVC': SVC(C=1, kernel='linear', random_state=42), 'SIFT_RF': RandomForestClassifier(n_estimators=1000, random_state=42), 'SIFT_KNN': KNeighborsClassifier(n_neighbors=1), 'ORB_KMEANS': MiniBatchKMeans(n_clusters=1000), 'ORB_SVC': SVC(C=1, kernel='linear', random_state=42), 'ORB_RF': RandomForestClassifier(n_estimators=1000, random_state=42), 'ORB_KNN': KNeighborsClassifier(n_neighbors=1), 'HARRIS_KMEANS': KMeans(n_clusters=100), 'HARRIS_SVC': SVC(C=1, kernel='linear'), 'HARRIS_RF': RandomForestClassifier(random_state=42), 'HARRIS_KNN': KNeighborsClassifier(n_neighbors=1)}
Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://5b8baba5e9547cbf7a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/home/skywalker/code/University/COMP702/Comp702_Project/.venv/lib/python3.12/site-packages/gradio/queueing.py", line 532, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/skywalker/code/University/COMP702/Comp702_Project/.venv/lib/python3.12/site-packages/gradio/route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/skywalker/code/University/COMP702/Comp702_Project/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1923, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/skywalker/code/University/COMP702/Comp702_Project/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1509, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
